In [16]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

#************Load data*******************************
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images     # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images       # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)



Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [39]:
#******Model function for implementing CNN*********
def cnn_funct(features,labels,mode):
    input_layer = tf.reshape(features["x"],[-1,28,28,1])

#******Convolutional Layer 1*********
    conv_1 = tf.layers.conv2d(inputs = input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

#******Pooling Layer 1****************
    pool_1 = tf.layers.max_pooling2d(inputs = conv_1, pool_size=[2, 2], strides=2)

#******Convolutional Layer 2*********
    conv_2 = tf.layers.conv2d(inputs = pool_1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

#******Pooling Layer 2****************
    pool_2 = tf.layers.max_pooling2d(inputs = conv_2, pool_size=[2, 2], strides=2)

#******Dense Layer********************
    pool2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

#******Logits Layer*******************
    logits = tf.layers.dense(inputs=dropout, units=10)

#******Generate predictions (for PREDICT and TEST mode) and add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
    predictions = {"classes": tf.argmax(input=logits, axis=1),
                   "probabilities": tf.nn.softmax(logits, name="softmax_tensor")}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

#******Calculate Loss (for both TRAIN and TEST modes)***********
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

#******Configure the Training Op (for TRAIN mode)***************
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

#*******Add evaluation metrics (for TEST mode)******************
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)    

In [40]:
#*************Create the model estimator which performs classification***********************************
#def model_classifier(model_funct):
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_funct, model_dir="./Project 3- Classification/mnist_convnet_model")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './Project 3- Classification/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1183f1dd8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
#********Set up logging to log the probability values from the softmax layer of our CNN*************
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)


In [34]:
#********Train the model****************************
def train_model(x):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
    mnist_classifier.train(input_fn=train_input_fn,steps=1000,hooks=[logging_hook])
    

In [ ]:
#*********Test the model and print results***********
def test_model(x):
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)
    test_results = mnist_classifier.evaluate(input_fn=test_input_fn)
    print(test_results)
    

In [35]:
train_model(train_data)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./Project 3- Classification/mnist_convnet_model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into ./Project 3- Classification/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.10169162  0.10336597  0.09183309  0.09700543  0.09784395  0.09944168
   0.1100842   0.08577216  0.11004721  0.10291471]
 [ 0.10558946  0.09370977  0.09087056  0.08843683  0.11135218  0.10945585
   0.10412943  0.10187075  0.10216153  0.09242368]
 [ 0.10221431  0.10013407  0.08665134  0.09287501  0.09955365  0.10045687
   0.10988819  0.10138862  0.12775452  0.07908349]
 [ 0.10686713  0.09996077  0.09171556  0.09449434  0.08969924  0.09608968
   0.11328333  0.09876076  0.10747797  0.1016512 ]
 [ 0.09978243  0.10245714  0.10095194  0.09535217  0.09047897  0.10506165
   0.10949478  0.09764794  0.10257686  0.09619608]
 [ 0.11289784  0.10556711  0.1070798   0.08918618  0.09845761  0.09060322
   0.10196517  0

INFO:tensorflow:loss = 2.29754, step = 2
INFO:tensorflow:probabilities = [[ 0.10153373  0.09890896  0.11579007  0.09874226  0.09335891  0.09408894
   0.10151529  0.10440874  0.10154932  0.09010375]
 [ 0.11187528  0.09775905  0.09530138  0.09936054  0.10834926  0.09157617
   0.11291625  0.08460609  0.10770551  0.09055047]
 [ 0.09904101  0.11661553  0.10538991  0.08426914  0.09722015  0.09926853
   0.11329201  0.09687322  0.09701201  0.09101856]
 [ 0.12227814  0.10235681  0.09750785  0.0951182   0.09347654  0.08973311
   0.10852469  0.09251992  0.10360239  0.09488235]
 [ 0.10235776  0.10060742  0.10422502  0.09213965  0.10136327  0.0953428
   0.11168926  0.09122149  0.11825593  0.08279745]
 [ 0.10812278  0.11124682  0.09623186  0.08630607  0.09368656  0.10747389
   0.11931992  0.09412132  0.08676733  0.09672347]
 [ 0.10472732  0.10083304  0.09087548  0.0902859   0.10208811  0.10282236
   0.09862152  0.10146835  0.11699992  0.09127799]
 [ 0.10479259  0.10350421  0.10026807  0.09324598  0.

INFO:tensorflow:global_step/sec: 2.3399
INFO:tensorflow:probabilities = [[ 0.10599661  0.10059265  0.09278829  0.09340074  0.10003493  0.0945351
   0.1040629   0.09740014  0.11491371  0.0962749 ]
 [ 0.10068036  0.0972997   0.10063086  0.09878592  0.09435564  0.10238653
   0.10730888  0.09060529  0.11489136  0.09305549]
 [ 0.09721259  0.09677527  0.0971633   0.09617225  0.09229644  0.09940888
   0.11063245  0.10306876  0.11383928  0.09343083]
 [ 0.10833891  0.0939371   0.10096358  0.0983323   0.09933482  0.09558525
   0.09764508  0.09539919  0.10548821  0.10497556]
 [ 0.10523692  0.09577595  0.10136955  0.09478844  0.1001473   0.09295638
   0.10986144  0.09538071  0.11192127  0.0925621 ]
 [ 0.10915753  0.1021221   0.09958621  0.09325313  0.09863922  0.09678084
   0.10593073  0.10031701  0.10571726  0.08849595]
 [ 0.10016051  0.09902731  0.09577354  0.09803654  0.09635796  0.09957214
   0.10637771  0.10022728  0.10938615  0.09508096]
 [ 0.09313001  0.10032542  0.10117442  0.10088702  0.1

INFO:tensorflow:loss = 2.29212, step = 102 (42.744 sec)
INFO:tensorflow:probabilities = [[ 0.11303093  0.09490258  0.10465738  0.09565166  0.11570902  0.08372175
   0.11073308  0.07885139  0.11670821  0.08603402]
 [ 0.1167352   0.09884129  0.10511309  0.09742848  0.09450175  0.09552673
   0.10211311  0.09456854  0.10494775  0.09022412]
 [ 0.11035476  0.09330598  0.11232608  0.08899369  0.11102549  0.08729807
   0.11478368  0.08947982  0.10758995  0.08484249]
 [ 0.1004246   0.0998655   0.10404503  0.09436074  0.08698923  0.10824838
   0.101238    0.10188903  0.10838217  0.0945573 ]
 [ 0.09660537  0.10299441  0.10184185  0.09114972  0.10008546  0.10374725
   0.0990233   0.08869235  0.1215043   0.09435594]
 [ 0.11500799  0.10187337  0.11358479  0.08639298  0.0974565   0.10381741
   0.10155733  0.09294886  0.09692194  0.09043884]
 [ 0.10935544  0.09520994  0.09691025  0.09140884  0.10469057  0.09713414
   0.11424288  0.09429867  0.10982937  0.0869199 ]
 [ 0.09944447  0.09365882  0.10269843

INFO:tensorflow:global_step/sec: 2.55309
INFO:tensorflow:probabilities = [[ 0.10331511  0.10715763  0.10168652  0.0957351   0.09574157  0.09239671
   0.10018477  0.10589155  0.10427491  0.0936161 ]
 [ 0.09894387  0.09514729  0.08765722  0.10131461  0.1046136   0.08853975
   0.09768171  0.10507661  0.13200843  0.08901688]
 [ 0.09572998  0.09298613  0.0937081   0.09734899  0.10258012  0.10676624
   0.1009535   0.10550064  0.1054072   0.09901907]
 [ 0.11073637  0.09920143  0.1005583   0.09617968  0.08668797  0.10104774
   0.11076584  0.0904045   0.11131196  0.09310618]
 [ 0.10126903  0.09824231  0.09355823  0.10041913  0.0924618   0.09751044
   0.11516735  0.09583811  0.11449542  0.0910382 ]
 [ 0.10877759  0.09083155  0.09282444  0.09173063  0.11447293  0.09648933
   0.09855929  0.10286845  0.11487382  0.08857206]
 [ 0.09751405  0.10550396  0.11020742  0.09221392  0.09502515  0.09417453
   0.10855444  0.09993462  0.09983169  0.0970403 ]
 [ 0.11143163  0.0980983   0.09096186  0.10023595  0

...

INFO:tensorflow:global_step/sec: 2.92724
INFO:tensorflow:probabilities = [[ 0.10640276  0.08331523  0.11000734  0.10968891  0.0972835   0.08521113
   0.13295507  0.07421177  0.10857292  0.09235138]
 [ 0.12708592  0.07098318  0.10704549  0.06950931  0.1236078   0.10999408
   0.10765827  0.10247858  0.10128944  0.08034795]
 [ 0.08960845  0.0785182   0.06904418  0.10859124  0.11562899  0.06902266
   0.11664502  0.12652746  0.11827128  0.10814258]
 [ 0.08532314  0.13710192  0.10054682  0.09675746  0.0826261   0.08170938
   0.09557137  0.10707822  0.10710056  0.10618512]
 [ 0.1429572   0.07112245  0.10718728  0.09586953  0.08387524  0.08037962
   0.15436547  0.05401073  0.1052582   0.10497431]
 [ 0.09054311  0.10078748  0.11320128  0.09469593  0.09965819  0.07551298
   0.09007162  0.10406912  0.13078232  0.10067792]
 [ 0.10101236  0.10439554  0.0876276   0.09492446  0.08588368  0.11296872
   0.11618631  0.07991765  0.12633368  0.09074991]
 [ 0.23104782  0.06705541  0.10091171  0.09882244  0

INFO:tensorflow:loss = 2.11678, step = 802 (34.203 sec)
INFO:tensorflow:probabilities = [[ 0.16992676  0.06849064  0.11046702  0.08716355  0.08785434  0.11658384
   0.11534382  0.07212846  0.09302838  0.07901328]
 [ 0.10766182  0.1028012   0.10773467  0.09793155  0.11555966  0.07295336
   0.10048299  0.10156825  0.09657655  0.0967299 ]
 [ 0.19840701  0.07594013  0.11736059  0.09364714  0.07286514  0.09318306
   0.14166059  0.07043886  0.08076157  0.05573589]
 [ 0.08483172  0.06286018  0.13271756  0.13668038  0.08112924  0.0910437
   0.12865406  0.076226    0.11759605  0.08826108]
 [ 0.10075025  0.09446465  0.08054611  0.10117886  0.12911342  0.08396047
   0.09519588  0.11008464  0.09834959  0.10635617]
 [ 0.09658697  0.07653108  0.08762598  0.09390532  0.10824166  0.09567653
   0.10192902  0.12579098  0.11172248  0.10199001]
 [ 0.18837604  0.05796184  0.09789511  0.10162944  0.08807559  0.07916781
   0.083258    0.08327073  0.10605709  0.11430844]
 [ 0.09339199  0.07846845  0.07232784 

INFO:tensorflow:global_step/sec: 2.57434
INFO:tensorflow:probabilities = [[ 0.10778373  0.08432771  0.08235887  0.12386655  0.08557627  0.10655804
   0.10170199  0.08431841  0.12784766  0.09566073]
 [ 0.14119132  0.06392036  0.09271441  0.10748796  0.10084697  0.12104275
   0.09189591  0.10858922  0.09664319  0.07566795]
 [ 0.07861745  0.13630295  0.08562723  0.09799956  0.08119058  0.10096279
   0.10648775  0.10168821  0.11543232  0.09569114]
 [ 0.11664627  0.06670255  0.0847936   0.09714142  0.1280151   0.08570378
   0.13686331  0.10374933  0.09904823  0.08133633]
 [ 0.10862425  0.09443787  0.08052853  0.0969189   0.10568494  0.07325149
   0.10880858  0.10254554  0.10575251  0.12344734]
 [ 0.12182989  0.0581038   0.10626931  0.0721729   0.11877792  0.07952248
   0.11672359  0.10376827  0.11632116  0.10651069]
 [ 0.17175326  0.06269805  0.11415     0.09918386  0.12789182  0.07170023
   0.10788524  0.05841574  0.09086756  0.09545422]
 [ 0.17419046  0.04862399  0.1207879   0.13324319  0

INFO:tensorflow:loss = 2.06351, step = 902 (38.810 sec)
INFO:tensorflow:probabilities = [[ 0.11470418  0.09997104  0.10605014  0.09368294  0.09547035  0.10349628
   0.09394448  0.0855383   0.11574226  0.09140011]
 [ 0.10338142  0.0895907   0.06667635  0.07238609  0.12425804  0.07300621
   0.11561321  0.10087658  0.14310786  0.1111035 ]
 [ 0.08588913  0.07389862  0.09032761  0.09353334  0.09897858  0.09083496
   0.16866793  0.0704172   0.12786324  0.09958945]
 [ 0.08178212  0.14582492  0.10277486  0.09097304  0.07985073  0.08184799
   0.09002208  0.10670178  0.11203167  0.1081908 ]
 [ 0.12323859  0.05869306  0.15578032  0.11050665  0.14464004  0.06310055
   0.09457078  0.04806959  0.11251371  0.08888673]
 [ 0.12255792  0.08788814  0.14703746  0.09734369  0.06211455  0.07315622
   0.17557085  0.06169111  0.08431602  0.08832397]
 [ 0.08626612  0.05902691  0.11359464  0.1340576   0.09581336  0.09727821
   0.12369332  0.0748108   0.11770547  0.09775349]
 [ 0.14069581  0.0592108   0.09163173

INFO:tensorflow:Saving checkpoints for 1001 into ./Project 3- Classification/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.91302.


In [42]:
test_model(test_data)

INFO:tensorflow:Starting evaluation at 2017-11-19-18:27:58
INFO:tensorflow:Restoring parameters from ./Project 3- Classification/mnist_convnet_model/model.ckpt-1001
INFO:tensorflow:Finished evaluation at 2017-11-19-18:28:11
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.6657, global_step = 1001, loss = 1.93851
{'accuracy': 0.66570002, 'loss': 1.9385058, 'global_step': 1001}
